#Task-3 w entities and w/o KG 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%rm -r "/content/evaluation"

rm: cannot remove '/content/evaluation': No such file or directory


In [ ]:
!git clone https://github.com/ncg-task/evaluation-phase1 "/content/evaluation1/"

Cloning into '/content/evaluation1'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (490/490), done.
remote: Compressing objects: 100% (483/483), done.
remote: Total 490 (delta 7), reused 488 (delta 5), pack-reused 0
Receiving objects: 100% (490/490), 2.50 MiB | 6.20 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
!git clone https://github.com/ncg-task/evaluation-phase2 "/content/sent1"

Cloning into '/content/sent1'...
remote: Enumerating objects: 334, done.
remote: Counting objects: 100% (334/334), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 334 (delta 5), reused 332 (delta 3), pack-reused 0
Receiving objects: 100% (334/334), 21.40 KiB | 5.35 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
!git clone https://github.com/ncg-task/evaluation-phase2-part2 "/content/entity1"

Cloning into '/content/entity1'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (326/326), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 326 (delta 2), reused 326 (delta 2), pack-reused 0
Receiving objects: 100% (326/326), 202.11 KiB | 1.14 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
# Test dataset reading
import os
from shutil import copyfile
output_dir = "/content/evaluation1/"
test1_input_dir = "/content/sent1/"
test2_input_dir = "/content/entity1/"
test_list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
test1_file_name_list = []
t_files = 0
for fls in test_list_of_folders:
  count=0
  for i in os.listdir(test1_input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(test1_input_dir + fls + '/' + str(i)):
      if files.endswith("sentences.txt"):
        copyfile(test1_input_dir + fls + '/' + str(i) + '/' + files, output_dir + fls + '/' + str(i) + '/' + files)
        t_files=t_files+1
    test1_file_name_list.append(fls + '/' + str(i))
  print("completed sent",fls,count)

test2_file_name_list = []
for fls in test_list_of_folders:
  for i in os.listdir(test2_input_dir + fls + '/'):
    for files in os.listdir(test2_input_dir + fls + '/' + str(i)):
      if files.endswith("entities.txt"):
        copyfile(test2_input_dir + fls + '/' + str(i) + '/' + files, output_dir + fls + '/' + str(i) + '/' + files)
    test2_file_name_list.append(fls + '/' + str(i))
  print("completed entity",fls)

completed sent constituency_parsing 9
completed sent coreference_resolution 10
completed sent data-to-text_generation 7
completed sent dependency_parsing 9
completed sent document_classification 21
completed sent entity_linking 17
completed sent face_alignment 19
completed sent face_detection 22
completed sent hypernym_discovery 9
completed sent natural_language_inference 32
completed entity constituency_parsing
completed entity coreference_resolution
completed entity data-to-text_generation
completed entity dependency_parsing
completed entity document_classification
completed entity entity_linking
completed entity face_alignment
completed entity face_detection
completed entity hypernym_discovery
completed entity natural_language_inference


In [ ]:
%cp -R "/content/evaluation1" "/content/drive/MyDrive/sub3_ph22"

In [ ]:
test_input_dir = "/content/drive/MyDrive/sub3_ph22/"
# Test dataset reading
import os
from shutil import copyfile
test_list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
t_files = 0
for fls in test_list_of_folders:
  for i in os.listdir(test_input_dir + fls + '/'):
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
      if files.endswith("entities.txt"):
        t_files=t_files+1
  print("completed",fls,t_files)

completed constituency_parsing 9
completed coreference_resolution 19
completed data-to-text_generation 26
completed dependency_parsing 35
completed document_classification 56
completed entity_linking 73
completed face_alignment 92
completed face_detection 114
completed hypernym_discovery 123
completed natural_language_inference 155


#Task-3 (entities)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers
########### evaluating model
import numpy as np
from sklearn import metrics
import transformers
import torch
import time
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import random

### Training label formation
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

     |████████████████████████████████| 1.8MB 8.5MB/s 
     |████████████████████████████████| 890kB 37.4MB/s 
     |████████████████████████████████| 2.9MB 57.5MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
output_dir = "/content/drive/MyDrive/sub3_ph22/"
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 
#self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
class SCIBERTClass(torch.nn.Module):
    def __init__(self):
        super(SCIBERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', output_hidden_states=True)
        self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True, dropout=0.1)
        self.l2 = torch.nn.Dropout(0.2)
        self.l3 = torch.nn.Linear(800, 400)
        self.l4 = torch.nn.Linear(400,100)
        self.l5 = torch.nn.Linear(100,10)
    
    def forward(self, ids, mask, token_type_ids, lengths):
        encoded_layers = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths, batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
        output_hidden = self.l2(output_hidden)            # no size change
        output_2 = self.l3(output_hidden)
        output_2 = torch.nn.ReLU()(output_2)
        output_4 = self.l4(output_2)
        output_5 = self.l5(output_4)
        return output_5

model = SCIBERTClass()
model.to(device)
model = torch.load("/content/drive/MyDrive/phase-2-task3/15-entity-task3.pt")
model.eval()
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
MAX_LEN = 512

In [ ]:
infoname_test_to_idx = {'approach.txt': 0, 'model.txt': 1, 'dataset.txt': 2, 
        'experimental-setup.txt': 3, 'hyperparameters.txt': 4,  'baselines.txt': 5, 'results.txt': 6, 
        'tasks.txt': 7, 'experiments.txt': 8, 'ablation-analysis.txt': 9}

infoname_idx_to_test = {v: k for k, v in infoname_test_to_idx.items()}

infoname_test_to_first_triplet = {'approach.txt': 'Approach', 'model.txt': 'Model', 'dataset.txt': 'Dataset', 
        'experimental-setup.txt': 'Experimental setup', 'hyperparameters.txt': 'Hyperparameters',  'baselines.txt': 'Baselines', 'results.txt': 'Results', 
        'tasks.txt': 'Tasks', 'experiments.txt': 'Experiments', 'ablation-analysis.txt': 'Ablation analysis'} #research problem and code has no first triplet

In [ ]:
import os
#####Test model
#test_input_dir = "/content/drive/My Drive/Dev_v2/"
test_input_dir = "/content/drive/MyDrive/sub3_ph22/"
# test_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
#          "relation-classification", "text-classification"]
test_list_of_folders = list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
test_input_stanza_list = []
test_input_sent_num_list = []
test_file_name_list = []
test_input_entity_list = []
test_input_triple_list = []
test_input_unit_name = []
test_total_phrases = 0
test_triplet_list = []
test_total_triplets = 0
for fls in test_list_of_folders:
  for i in os.listdir(test_input_dir + fls + '/'):
    triple_list = []
    unit_name = []
    triplet_lines_list = []
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        test_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        test_input_sent_num_list.append(list(map(int, sentence_num_list)))
      if files.endswith("entities.txt"):
        entities_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,map(lambda x:x,entities_file.read().splitlines()))) # filter empty strings and split into lines
        test_input_entity_list.append(entities_list)
    ### start embedd function for the info unit prediction
    entity_phrase_only = [j.lower().split('\t') for j in entities_list]
    entity_phrase_only.sort(key=lambda x: (int(x[0]),int(x[1])))
    entity_ph_only_string = ' '.join([j[3] for j in entity_phrase_only])
    word_tokens = word_tokenize(entity_ph_only_string)
    filter_ph_only_string = ' '.join([w for w in word_tokens if not w in stop_words])
    print("output", filter_ph_only_string)
    output_info_folder = os.path.join(test_input_dir,fls,str(i),'triples')
    if not os.path.exists(output_info_folder):
      os.makedirs(output_info_folder)
    with torch.no_grad():
      test_input = tokenizer.encode_plus(filter_ph_only_string.lower(), None, add_special_tokens=True, 
                                         max_length= MAX_LEN, pad_to_max_length=True, 
                                         return_token_type_ids=True, truncation=True, 
                                         return_length = True,return_tensors="pt")
      test_ids = test_input['input_ids'].to(device, dtype = torch.long)
      test_mask = test_input['attention_mask'].to(device, dtype = torch.long)
      test_token_type_ids = test_input['token_type_ids'].to(device, dtype = torch.long)
      test_lengths = test_input['length']#.squeeze(dim=1)#.to(device, dtype = torch.long)
      #print("shape",test_ids.shape, test_mask.shape, test_token_type_ids.shape, test_lengths)
      outputs = model(test_ids, test_mask, test_token_type_ids, test_lengths)
      output_idx = np.array(torch.sigmoid(outputs).cpu().detach().numpy().tolist()) >=0.5
      print("output class", output_idx)
      # for i in range(output_idx.shape[1]):
      #   if output_idx[0][i] == True:
      #     output_file=open(os.path.join(test_input_dir,fls,str(i),'triples',infoname_idx_to_test[i]),"a")
      #     output_file.write('(Contribution||has||'+infoname_test_to_first_triplet[infoname_idx_to_test[i]]+')'+"\n")
  print("completed",fls,test_total_phrases)
    ### Training label formation




  #   t_entity_list = [j.split('\t') for j in entities_list]
  #   t_entity_list.sort(key=lambda x: (int(x[0]),int(x[1]))) #reverse = True)
  #   t_triplet_formed = []
  #   for t5 in range(len(t_entity_list)):
  #     try:
  #       if((t_entity_list[t5][0]==t_entity_list[t5+1][0])  and (t_entity_list[t5][0]==t_entity_list[t5+2][0])):
  #         t_triplet_formed = t_triplet_formed + [[t_entity_list[t5][3], t_entity_list[t5+1][3], t_entity_list[t5+2][3]]]
  #     except Exception as e:
  #       continue
  #   ###triplet end
  #   t_triplet_formed_string = [' '.join(x) for x in t_triplet_formed]
  #   #print("size",len(t_triplet_formed_string))
  #   output_info_folder = os.path.join(test_input_dir,fls,str(i),'triples')
  #   if not os.path.exists(output_info_folder):
  #     os.makedirs(output_info_folder)
  #   with torch.no_grad():
  #     for t6 in range(len(t_triplet_formed_string)): #lowercase the string input to classifier
  #       test_input = tokenizer.encode_plus(t_triplet_formed_string[t6].lower(), None, add_special_tokens=True, 
  #                                          max_length= MAX_LEN, pad_to_max_length=True, 
  #                                          return_token_type_ids=True, truncation=True, 
  #                                          return_length = True,return_tensors="pt")
  #       test_ids = test_input['input_ids'].to(device, dtype = torch.long)
  #       test_mask = test_input['attention_mask'].to(device, dtype = torch.long)
  #       test_token_type_ids = test_input['token_type_ids'].to(device, dtype = torch.long)
  #       test_lengths = test_input['length']#.squeeze(dim=1)#.to(device, dtype = torch.long)
  #       #print("shape",test_ids.shape, test_mask.shape, test_token_type_ids.shape, test_lengths)
  #       outputs = model(test_ids, test_mask, test_token_type_ids, test_lengths)
  #       _, output_idx = torch.max(outputs, dim =1)
  #       t_output_class = output_idx.cpu().detach().numpy().tolist()
  #       #print(t_output_class,'(' + '||'.join(t_triplet_formed[0])+')')
        
  #       #print("hbas",t_output_class[0])
  #       #for t7 in range(len(t_output_class)):
  #         #print(infoname_test_to_idx[t_output_class[t7]])
  #       output_file=open(os.path.join(output_info_folder, infoname_idx_to_test[t_output_class[0]]),"a")
  #       output_file.write('(' + '||'.join(t_triplet_formed[t6])+')'+"\n")
  #         ####Info classification end
  #         #### end embedd function for test
  #     test_file_name_list.append(fls + '/' + str(i))
  #   #if len(os.listdir(os.path.join(test_input_dir,fls,str(i),'triples'))) !=0:
  #   for file_add in os.listdir(test_input_dir + fls + '/' + str(i) + '/triples'):
  #     if file_add in list(infoname_test_to_first_triplet.keys()):
  #       output_file=open(os.path.join(test_input_dir,fls,str(i),'triples',file_add),"a")
  #       output_file.write('(Contribution||has||'+infoname_test_to_first_triplet[file_add]+')'+"\n")
  # print("completed",fls,test_total_phrases)

output recurrent neural network grammars recurrent neural network grammars ( rnng ) rnngs manipulates inductive bias rnngs test linguistic hypotheses begin ablation study discover importance composition function augment rnng composition function novel gated attention mechanism incorporate interpretability using ga - rnng investigating role individual heads play phrasal representation nonterminal category labels play rnng stack strongest ablations outperforms `` full `` rnng three data structures ablating stack gives worst among new results seen language modeling stack - rnng achieves best performance ablating stack harmful modeling syntax without explicit composition provides little benefit sequential lstm language model stack - results best published ptb results phrasestructure dependency parsing gated attention rnng clear model outperforms baseline rnng three structures achieves competitive performance strongest , stack - , rnng variant headedness phrases model higher overlap convers

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


output class [[False  True False False False False  True False False False]]
output syntactic constituency parsing constructed artificial dataset labelling large corpus berkeleyparser trained sequence - - sequence model attention small human - annotated parsing dataset second artificial dataset consisting high - confidence parse trees measured agreement two parsers trained sequence - - sequence model attention used model 3 lstm layers 256 units layer call lstm + training small dataset additionally used 2 dropout layers lstm 1 lstm 2 lstm 2 lstm 3 embedding layer 90k vocabulary initialized randomly using pre-trained word - vector embeddings pre-trained skip - gram embeddings size 512 using word2vec 10b - word corpus single attention model gets 88.3 ensemble 5 lstm + a+d models achieves 90.5 matching single - model berkeleyparser wsj trained large high - confidence corpus single lstm + model achieves 92.5 outperforms best single model best ensemble result ensemble 5 lstm+ models improves

In [ ]:
import os
#####Test model
#test_input_dir = "/content/drive/My Drive/Dev_v2/"
test_input_dir = "/content/drive/MyDrive/sub3_ph22/"
# test_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
#          "relation-classification", "text-classification"]
test_list_of_folders = list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
test_input_stanza_list = []
test_input_sent_num_list = []
test_file_name_list = []
test_input_entity_list = []
test_input_triple_list = []
test_input_unit_name = []
test_total_phrases = 0
test_triplet_list = []
test_total_triplets = 0
for fls in test_list_of_folders:
  for i in os.listdir(test_input_dir + fls + '/'):
    triple_list = []
    unit_name = []
    triplet_lines_list = []
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        test_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        test_input_sent_num_list.append(list(map(int, sentence_num_list)))
      if files.endswith("entities.txt"):
        entities_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,map(lambda x:x,entities_file.read().splitlines()))) # filter empty strings and split into lines
        test_input_entity_list.append(entities_list)
    ### start embedd function for the info unit prediction
    entity_phrase_only = [j.lower().split('\t') for j in entities_list]
    entity_phrase_only.sort(key=lambda x: (int(x[0]),int(x[1])))
    entity_ph_only_string = ' '.join([j[3] for j in entity_phrase_only])
    word_tokens = word_tokenize(entity_ph_only_string)
    filter_ph_only_string = ' '.join([w for w in word_tokens if not w in stop_words])
    #print("output", filter_ph_only_string)
    output_info_folder = os.path.join(test_input_dir,fls,str(i),'triples')
    if not os.path.exists(output_info_folder):
      os.makedirs(output_info_folder)
    with torch.no_grad():
      test_input = tokenizer.encode_plus(filter_ph_only_string.lower(), None, add_special_tokens=True, 
                                         max_length= MAX_LEN, pad_to_max_length=True, 
                                         return_token_type_ids=True, truncation=True, 
                                         return_length = True,return_tensors="pt")
      test_ids = test_input['input_ids'].to(device, dtype = torch.long)
      test_mask = test_input['attention_mask'].to(device, dtype = torch.long)
      test_token_type_ids = test_input['token_type_ids'].to(device, dtype = torch.long)
      test_lengths = test_input['length']#.squeeze(dim=1)#.to(device, dtype = torch.long)
      #print("shape",test_ids.shape, test_mask.shape, test_token_type_ids.shape, test_lengths)
      outputs = model(test_ids, test_mask, test_token_type_ids, test_lengths)
      output_idx = np.array(torch.sigmoid(outputs).cpu().detach().numpy().tolist()) >=0.5
      #print("output class", output_idx)
      for i in range(output_idx.shape[1]):
        if output_idx[0][i] == True:
          #print(infoname_idx_to_test[i])
          output_file=open(os.path.join(output_info_folder,infoname_idx_to_test[i]),"w")
          output_file.write('(Contribution||has||'+infoname_test_to_first_triplet[infoname_idx_to_test[i]]+')'+"\n")
  print("completed",fls,test_total_phrases)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


completed constituency_parsing 0
completed coreference_resolution 0
completed data-to-text_generation 0
completed dependency_parsing 0
completed document_classification 0
completed entity_linking 0
completed face_alignment 0
completed face_detection 0
completed hypernym_discovery 0
completed natural_language_inference 0


#TASK-3 using KG-BERT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
infoname_to_idx = {'approach.txt': 0, 'model.txt': 1, 'dataset.txt': 2, 
        'experimental-setup.txt': 3, 'hyperparameters.txt': 4,  'baselines.txt': 5, 'results.txt': 6, 
        'tasks.txt': 7, 'experiments.txt': 8, 'ablation-analysis.txt': 9}
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
!git clone https://github.com/ncg-task/training-data.git "/content/train"
!git clone https://github.com/ncg-task/trial-data.git "/content/valid"

fatal: destination path '/content/train' already exists and is not an empty directory.
fatal: destination path '/content/valid' already exists and is not an empty directory.


In [ ]:
# Loading the Training Data
input_dir = "/content/train/"
import os
import torch
# list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
#                   "relation-classification","text-classification"]
list_of_folders = ['phrase_grounding', 'question_answering',
 'sentence_compression', 'question_similarity',
 'question_generation', 'sentence_classification',
 'text_summarization', 'paraphrase_generation',
 'smile_recognition', 'prosody_prediction',
 'natural_language_inference', 'semantic_parsing',
 'sentiment_analysis', 'temporal_information_extraction',
 'topic_models', 'relation_extraction',
 'semantic_role_labeling', 'query_wellformedness',
 'negation_scope_resolution', 'sarcasm_detection',
 'passage_re-ranking', 'text-to-speech_synthesis',
 'text_generation', 'part-of-speech_tagging']
input_stanza_list = []
input_sent_num_list = []
input_entity_list = []
file_name_list = []
input_triplet_list = []
total_triplets = 0
for fls in list_of_folders:
  for i in os.listdir(input_dir + fls + '/'):
    triplet_lines_list =[]
    for files in os.listdir(input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None,map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        input_sent_num_list.append(list(map(int, sentence_num_list)))
      if files.endswith("entities.txt"):
        entities_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,map(lambda x:x.lower(),entities_file.read().splitlines()))) # filter empty strings and split into lines
        input_entity_list.append(entities_list)
    file_name_list.append(fls + '/' + str(i))
    for files in os.listdir(input_dir + fls + '/' + str(i) + '/triples'):
      #unit_name.append(files)
      info_unit_file = open(input_dir + fls + '/' + str(i) + '/triples/' + files, "r")
      if (files != 'research-problem.txt') and (files!='code.txt'):
        # info_unit_list = list(filter(None,map(lambda x:x.lower(), info_unit_file.read().lower().splitlines()))) # filter empty strings and split into lines
        # triple_list.extend(info_unit_list[1:])  # a list for each article of triples (triples of all info units of a article)
        # list_unit_corres_triples = [infoname_to_idx[files]]*len(info_unit_list[1:]) # list for assigning information units corresponding to each triple
        # unit_name.extend(list_unit_corres_triples)
      ####### start insert code
        triplet_file = open(input_dir + fls + '/' + str(i) + '/' + 'triples' + '/' + files)
        triplet_lines = (triplet_file.read()).lower()
        triplet_lines_string = list(filter(None,triplet_lines.splitlines()))
        triplets_broken_into_three = list(map(lambda x: x[1:-1].split("||")[1], triplet_lines_string)) 
        triplet_lines_list = triplets_broken_into_three[1:] + triplet_lines_list
    input_triplet_list.append(triplet_lines_list)
    total_triplets += len(triplet_lines_list)
    ###### finish insertion
  print("completed",fls,total_triplets)

completed phrase_grounding 96
completed question_answering 717
completed sentence_compression 955
completed question_similarity 997
completed question_generation 1072
completed sentence_classification 1354
completed text_summarization 2341
completed paraphrase_generation 2507
completed smile_recognition 2557
completed prosody_prediction 2655
completed natural_language_inference 10016
completed semantic_parsing 10180
completed sentiment_analysis 14348
completed temporal_information_extraction 14432
completed topic_models 14473
completed relation_extraction 15531
completed semantic_role_labeling 15836
completed query_wellformedness 15866
completed negation_scope_resolution 15952
completed sarcasm_detection 16077
completed passage_re-ranking 16194
completed text-to-speech_synthesis 16372
completed text_generation 16771
completed part-of-speech_tagging 17231


In [ ]:
input_triplet_list[1]

['for',
 'has',
 'has',
 'compared to',
 'on',
 'on',
 'has',
 'prefers',
 'is',
 'to achieve',
 'that consist of',
 'has',
 'maintains',
 'across',
 'shows',
 'of encoding',
 'has',
 'outperforms',
 'by',
 'has',
 'delivers',
 'see that',
 'offers',
 'on',
 'on',
 'has',
 'outperform',
 'across',
 'has',
 'showing',
 'adapt',
 'to process and score',
 'for',
 'construct',
 'has',
 'deterministically converted to',
 'to extract',
 'from',
 'has',
 'to',
 'of',
 'has',
 'to process',
 'has',
 'use',
 'to encode',
 'in',
 'has',
 'include',
 'directly computes',
 'as',
 'of',
 'that does not use']

In [ ]:
##train label formation
train_entity_label=[]
train_entity_ph = []
for i in range(len(file_name_list)):
  t1_label=[0]*len(input_entity_list[i])
  entity_phr = [j.lower().split('\t') for j in input_entity_list[i]]
  entity_phr.sort(key=lambda x: (int(x[0]),int(x[1])))
  entity_ph_only = [j[3] for j in entity_phr]
  train_entity_ph.append(entity_ph_only)
  for k in range(len(entity_ph_only)):
    if entity_ph_only[k] in input_triplet_list[i]:
      t1_label[k]=1
  train_entity_label.append(t1_label)

In [ ]:
train_entity_label[1][:20]

[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0]

In [ ]:
train_entity_ph[1][:20]

['knowledge base question answering',
 'knowledge base question answering ( qa )',
 'qa',
 'semantic parsing',
 'to',
 'problem',
 'of',
 'kb qa',
 'kb qa',
 'for',
 'each input question',
 'construct',
 'explicit structural semantic parse ( semantic graph )',
 'semantic parses',
 'deterministically converted to',
 'query',
 'to extract',
 'answers',
 'from',
 'kb']

In [ ]:
import os
##### validation dataset reading

val_input_dir = "/content/valid/"
val_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
         "relation-classification", "text-classification"]
val_input_stanza_list = []
val_input_sent_num_list = []
val_file_name_list = []
val_input_entity_list = []
valid_triplet_list = []
valid_total_triplets = 0
for fls in val_list_of_folders:
  for i in os.listdir(val_input_dir + fls + '/'):
    triplet_lines_list = []
    for files in os.listdir(val_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        val_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        val_input_sent_num_list.append(list(map(int, sentence_num_list)))
      if files.endswith("entities.txt"):
        entities_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,map(lambda x:x.lower(),entities_file.read().splitlines()))) # filter empty strings and split into lines
        val_input_entity_list.append(entities_list)
    val_file_name_list.append(fls + '/' + str(i))
    for files in os.listdir(val_input_dir + fls + '/' + str(i) + '/triples'):
      #unit_name.append(files)
      info_unit_file = open(val_input_dir + fls + '/' + str(i) + '/triples/' + files, "r")
      if (files != 'research-problem.txt') and (files!='code.txt'):
      ####### start insert code
        triplet_file = open(val_input_dir + fls + '/' + str(i) + '/' + 'triples' + '/' + files)
        triplet_lines = (triplet_file.read()).lower()
        triplet_lines_string = list(filter(None,triplet_lines.splitlines()))
        triplets_broken_into_three = list(map(lambda x: x[1:-1].split("||")[1], triplet_lines_string)) 
        triplet_lines_list = triplets_broken_into_three[1:] + triplet_lines_list
    valid_triplet_list.append(triplet_lines_list)
    valid_total_triplets += len(triplet_lines_list)
    ###### finish insertion
  print("completed",fls,valid_total_triplets)

completed machine-translation 485
completed named-entity-recognition 928
completed question-answering 1482
completed relation-classification 2023
completed text-classification 2593


In [ ]:
valid_triplet_list[1][:20]

['tasks',
 'outperforms',
 'for',
 'on',
 'tasks',
 'outperform',
 'for',
 'in the term of',
 'outperforms',
 'learned from',
 'outperforms',
 'by',
 'tasks',
 'improve the results of',
 'by',
 'in terms of',
 'outperforms',
 'on',
 'improves',
 'by']

In [ ]:
##valid label formation
valid_entity_label=[]
valid_entity_ph = []
for i in range(len(val_file_name_list)):
  t2_label=[0]*len(val_input_entity_list[i])
  entity_phr = [j.lower().split('\t') for j in val_input_entity_list[i]]
  entity_phr.sort(key=lambda x: (int(x[0]),int(x[1])))
  entity_ph_only = [j[3] for j in entity_phr]
  valid_entity_ph.append(entity_ph_only)
  for k in range(len(entity_ph_only)):
    if entity_ph_only[k] in valid_triplet_list[i]:
      t2_label[k]=1
  valid_entity_label.append(t2_label)

In [ ]:
print(valid_entity_label[1][:20])
print(valid_entity_ph[1][:20])

[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0]
['frequency - agnostic word representation', 'word embeddings learned in several tasks are biased towards word frequency', 'learned embeddings of rare words and popular words behave differently', 'embeddings of rare words and popular words actually lie in different subregions of the space', 'different behaviors of the embeddings of popular words and rare words are problematic', 'to learn', 'frequency - agnostic word embedding ( frage )', 'minimize', 'task - specific loss', 'by optimizing', 'task - specific parameters', 'together with', 'word embeddings', 'introduce', 'another discriminator', 'a word embedding', 'input', 'classifies', 'popular / rare word', 'discriminator']


In [ ]:
## single list conversion


In [ ]:
### Flatten train and valid dataset
from collections import Counter
train_ph_list = [j for sub in train_entity_ph for j in sub]
train_label_list = [j for sub in train_entity_label for j in sub]
valid_ph_list = [j for sub in valid_entity_ph for j in sub]
valid_label_list = [j for sub in valid_entity_label for j in sub]
print("Train all entities size", len(train_ph_list), len(train_label_list))
print("Valid all entities size", len(valid_ph_list), len(valid_label_list))
train_ph_list, train_label_list = list(zip(*(set(zip(train_ph_list,train_label_list)))))
valid_ph_list, valid_label_list = list(zip(*(set(zip(valid_ph_list,valid_label_list)))))
print("Train all entities size", len(train_ph_list), len(train_label_list))
print("Valid all entities size", len(valid_ph_list), len(valid_label_list))

Train all entities size 30485 30485
Valid all entities size 4777 4777
Train all entities size 13745 13745
Valid all entities size 3161 3161


In [ ]:
list(Counter(train_label_list).values())

[11241, 2504]

In [ ]:
!pip install -q transformers

In [ ]:
########### Pre-processing complete for train and valid. 
########### Start model building
import numpy as np
from sklearn import metrics
import transformers
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
##### Parameters for the model
EPOCHS = 10
LEARNING_RATE = 2e-05
#tokenizer = AutoTokenizer.from_pretrained("allenai/biomed_roberta_base")
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')			
MAX_LEN = 25
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32

In [ ]:
#### Dataset Loader
class Triage(Dataset):
    def __init__(self, data_in,data_out, tokenizer, max_len):
        self.len = len(data_in)
        self.data = data_in
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label = data_out
        
    def __getitem__(self, index):
        # title = str(self.data.TITLE[index])
        # title = " ".join(title.split())
        title = self.data[index]
        inputs = self.tokenizer.encode_plus(title,
            None,add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True,
            return_length = True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']
        lengths = inputs['length']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'lengths': torch.tensor(lengths, dtype=torch.long),
            'targets': torch.tensor(self.label[index], dtype=torch.long)
        } 
    
    def __len__(self):
      return self.len

In [ ]:
#### Train and Validation Loader 
training_set = Triage(train_ph_list,train_label_list, tokenizer, MAX_LEN)
train_params = {'batch_size': TRAIN_BATCH_SIZE,'shuffle': True,'num_workers': 0}
training_loader = DataLoader(training_set, **train_params)
valid_set = Triage(valid_ph_list,valid_label_list, tokenizer, MAX_LEN)
valid_params = {'batch_size': VALID_BATCH_SIZE,'shuffle': True,'num_workers': 0}
valid_loader = DataLoader(valid_set, **valid_params)

In [ ]:
# Create a SciBERT + Bi-LSTM model for binary classification
class SCIBERTClass(torch.nn.Module):
    def __init__(self):
        super(SCIBERTClass, self).__init__()
        #self.l1 = AutoModel.from_pretrained("allenai/biomed_roberta_base", output_hidden_states=True)
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased',  output_hidden_states=True)
        #self.lstm_hidden_size = lstm_hidden_size
        self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True, dropout=0.1)
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear( 800, 400)
        self.l4 = torch.nn.Linear(400,100) # check with layer with 30
        self.l5 = torch.nn.Linear(100,2)
    
    def forward(self, ids, mask, token_type_ids, lengths):
        encoded_layers = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        #bert_hidden_layer = bert_hidden_layer.permute(1, 0, 2)   #permute rotates the tensor. if tensor.shape = 3,4,5  tensor.permute(1,0,2), then tensor,shape= 4,3,5  (batch_size, sequence_length, hidden_size)
        enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths, batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
        output_hidden = self.l2(output_hidden)            # no size change
        output_2 = self.l3(output_hidden)
        output_2 = torch.nn.ReLU()(output_2)
        #output_3 = self.l2(output_2)
        output_4 = self.l4(output_2)
        #output_4 = torch.nn.ReLU()(output_4)
        output_5 = self.l5(output_4)
        return output_5

model = SCIBERTClass()
model.to(device)

SCIBERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [ ]:
#### Optimizer and Loss function
optimizer = torch.optim.AdamW(params = model.parameters(), lr=LEARNING_RATE)
c_weights = torch.tensor([list(Counter(train_label_list).values())[0], list(Counter(train_label_list).values())[1]], dtype=torch.float32).to(device)
c_weights = 1.0/(c_weights/c_weights.sum())
c_weights = c_weights/c_weights.sum()
loss_function = torch.nn.CrossEntropyLoss(weight = c_weights)

In [ ]:
def calculate_evaluation(f_targets, f_outputs):
  accuracy = metrics.accuracy_score(f_targets, f_outputs)
  eval = metrics.precision_recall_fscore_support(f_targets, f_outputs, average='micro')
  return accuracy, eval

In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    nb_tr_steps=0
    fin_targets = []
    fin_outputs = []
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        lengths = data['lengths'].squeeze(1)#.to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        #print("check shape",ids.shape,lengths.shape, targets.dtype)
        outputs = model(ids, mask, token_type_ids, lengths)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        nb_tr_steps += 1
        # if _%5000==0:
        #     print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        fin_targets.extend(targets.cpu().detach().numpy().tolist())
        _, output_idx = torch.max(outputs, dim =1)
        fin_outputs.extend(output_idx.cpu().detach().numpy().tolist())

    epoch_loss = tr_loss/nb_tr_steps
    accuracy, evaluate = calculate_evaluation(fin_targets, fin_outputs)
    #f_outputs = np.array(fin_outputs) >= 0.5
    #accuracy = metrics.accuracy_score(fin_targets, f_outputs)
    #f1_score_micro = metrics.f1_score(fin_targets, f_outputs, average='micro')
    #f1_score_macro = metrics.f1_score(fin_targets, f_outputs, average='macro')
    #print(f"Accuracy Score = {accuracy}")
    print(f"Train Epoch {epoch:02} | Loss:{epoch_loss} | Acc:{accuracy} | Prec:{evaluate[0]} | Rec:{evaluate[1]} | Mic-F1:{evaluate[2]}")
    return

In [ ]:
def validation(epoch, model, valid_loader):
  model.eval()
  fin_targets=[]
  fin_outputs=[]
  nb_tr_steps = 0
  tr_loss = 0
  with torch.no_grad():
    for _, data in enumerate(valid_loader, 0):
      ids = data['ids'].to(device, dtype = torch.long)
      mask = data['mask'].to(device, dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
      lengths = data['lengths'].squeeze(1)#.to(device, dtype = torch.long)
      targets = data['targets'].to(device, dtype = torch.long)
      outputs = model(ids, mask, token_type_ids, lengths)
      #temp_id_list = list(map(int, ids.cpu().numpy().tolist())) 
      # c1 = tokenizer.convert_ids_to_tokens(ids= ids.cpu().numpy().tolist()[0], skip_special_tokens= True)
      # print("check",c1)
      loss = loss_function(outputs, targets)
      tr_loss += loss.item()
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      _, output_idx = torch.max(outputs, dim =1)
      fin_outputs.extend(output_idx.cpu().detach().numpy().tolist())
      nb_tr_steps += 1
    epoch_loss = tr_loss/nb_tr_steps
    accuracy, evaluate = calculate_evaluation(fin_targets, fin_outputs)
    #print(fin_targets[:15],fin_outputs[:15])
    print(f"Valid Epoch {epoch:02} | Loss:{epoch_loss} | Acc:{accuracy} | Prec:{evaluate[0]} | Rec:{evaluate[1]} | Mic-F1:{evaluate[2]}")
  return

In [ ]:
#new model saved in new location
for epoch in range(EPOCHS):
    train(epoch)
    validation(epoch, model, valid_loader)
    if epoch==3:
      torch.save(model,f"/content/drive/MyDrive/phase-2-task3/{epoch}-predicate-task3.pt")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Train Epoch 00 | Loss:0.1849026964955725 | Acc:0.9422335394688978 | Prec:0.9422335394688978 | Rec:0.9422335394688978 | Mic-F1:0.9422335394688978
Valid Epoch 00 | Loss:0.36055578059055887 | Acc:0.9218601708320152 | Prec:0.9218601708320152 | Rec:0.9218601708320152 | Mic-F1:0.9218601708320152
Train Epoch 01 | Loss:0.1047591246469596 | Acc:0.9612222626409603 | Prec:0.9612222626409603 | Rec:0.9612222626409603 | Mic-F1:0.9612222626409603
Valid Epoch 01 | Loss:0.4630517481327659 | Acc:0.9304017715912686 | Prec:0.9304017715912686 | Rec:0.9304017715912686 | Mic-F1:0.9304017715912686
Train Epoch 02 | Loss:0.08770885153153782 | Acc:0.9673335758457621 | Prec:0.9673335758457621 | Rec:0.9673335758457621 | Mic-F1:0.9673335758457621
Valid Epoch 02 | Loss:0.4863064005933326 | Acc:0.9310344827586207 | Prec:0.9310344827586207 | Rec:0.9310344827586207 | Mic-F1:0.9310344827586207
Train Epoch 03 | Loss:0.07374544699248577 | Acc:0.9732266278646781 | Prec:0.9732266278646781 | Rec:0.9732266278646781 | Mic-F1:0

KeyboardInterrupt: ignored

# Inference for predicate

In [ ]:
!pip install -q transformers
########### evaluating model
import numpy as np
from sklearn import metrics
import transformers
import torch
import time
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import random
seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
output_dir = "/content/drive/MyDrive/sub5_ph22/"
# Create a SciBERT + Bi-LSTM model for binary classification
class SCIBERTClass(torch.nn.Module):
    def __init__(self):
        super(SCIBERTClass, self).__init__()
        #self.l1 = AutoModel.from_pretrained("allenai/biomed_roberta_base", output_hidden_states=True)
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased',  output_hidden_states=True)
        #self.lstm_hidden_size = lstm_hidden_size
        self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True, dropout=0.1)
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear( 800, 400)
        self.l4 = torch.nn.Linear(400,100) # check with layer with 30
        self.l5 = torch.nn.Linear(100,2)
    
    def forward(self, ids, mask, token_type_ids, lengths):
        encoded_layers = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        #bert_hidden_layer = bert_hidden_layer.permute(1, 0, 2)   #permute rotates the tensor. if tensor.shape = 3,4,5  tensor.permute(1,0,2), then tensor,shape= 4,3,5  (batch_size, sequence_length, hidden_size)
        enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths, batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
        output_hidden = self.l2(output_hidden)            # no size change
        output_2 = self.l3(output_hidden)
        output_2 = torch.nn.ReLU()(output_2)
        #output_3 = self.l2(output_2)
        output_4 = self.l4(output_2)
        #output_4 = torch.nn.ReLU()(output_4)
        output_5 = self.l5(output_4)
        return output_5

model = SCIBERTClass()
model.to(device)
model = torch.load("/content/drive/MyDrive/phase-2-task3/3-predicate-task3.pt")
model.eval()
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
MAX_LEN = 25

In [ ]:
infoname_test_to_idx = {'approach.txt': 0, 'model.txt': 1, 'dataset.txt': 2, 
        'experimental-setup.txt': 3, 'hyperparameters.txt': 4,  'baselines.txt': 5, 'results.txt': 6, 
        'tasks.txt': 7, 'experiments.txt': 8, 'ablation-analysis.txt': 9}

infoname_idx_to_test = {v: k for k, v in infoname_test_to_idx.items()}

infoname_test_to_first_triplet = {'approach.txt': 'Approach', 'model.txt': 'Model', 'dataset.txt': 'Dataset', 
        'experimental-setup.txt': 'Experimental setup', 'hyperparameters.txt': 'Hyperparameters',  'baselines.txt': 'Baselines', 'results.txt': 'Results', 
        'tasks.txt': 'Tasks', 'experiments.txt': 'Experiments', 'ablation-analysis.txt': 'Ablation analysis'} #research problem and code has no first triplet

In [ ]:
### Triple classifier model
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 
#self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
class SCIBERTClass(torch.nn.Module):
    def __init__(self):
        super(SCIBERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', output_hidden_states=True)
        self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True)
        self.l2 = torch.nn.Dropout(0.2)
        self.l3 = torch.nn.Linear(800, 400)
        self.l4 = torch.nn.Linear(400,100)
        self.l5 = torch.nn.Linear(100,10)
    
    def forward(self, ids, mask, token_type_ids, lengths):
        encoded_layers = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths, batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
        output_hidden = self.l2(output_hidden)            # no size change
        output_2 = self.l3(output_hidden)
        output_2 = torch.nn.ReLU()(output_2)
        output_4 = self.l4(output_2)
        output_5 = self.l5(output_4)
        return output_5

model_trip = SCIBERTClass()
model_trip.to(device)
model_trip = torch.load("/content/drive/MyDrive/modify_scibert-T1/1task3.pt")
model_trip.eval()
tokenizer_trip = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
MAX_LEN_trip = 50

In [26]:
import os
#####Test model
#test_input_dir = "/content/drive/My Drive/Dev_v2/"
test_input_dir = "/content/drive/MyDrive/sub5_ph22/"
# test_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
#          "relation-classification", "text-classification"]
test_list_of_folders = list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
test_input_stanza_list = []
test_input_sent_num_list = []
test_file_name_list = []
test_input_entity_list = []
test_input_triple_list = []
test_input_unit_name = []
test_total_phrases = 0
test_triplet_list = []
test_total_triplets = 0
for fls in test_list_of_folders:
  for i in os.listdir(test_input_dir + fls + '/'):
    triple_list = []
    unit_name = []
    triplet_lines_list = []
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        test_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        test_input_sent_num_list.append(list(map(int, sentence_num_list)))
      if files.endswith("entities.txt"):
        entities_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,map(lambda x:x,entities_file.read().splitlines()))) # filter empty strings and split into lines
        test_input_entity_list.append(entities_list)
      ### start embedd function for the combined task
    t_entity_list = [j.split('\t') for j in entities_list]
    t_entity_list.sort(key=lambda x: (int(x[0]),int(x[1]))) #reverse = True)
    t_entity_ph =  [j[3] for j in t_entity_list] # entity label in sorted order
    t_entity_label = [] # each entity corresponding label, 1= predicate and 0 = non-predicate
    #temp_list1 = []
    with torch.no_grad():
      for t5 in range(len(t_entity_ph)): #lowercase the string input to classifier
        test_input = tokenizer.encode_plus(t_entity_ph[t5].lower(), None, add_special_tokens=True, 
                                           max_length= MAX_LEN, pad_to_max_length=True, 
                                           return_token_type_ids=True, truncation=True, 
                                           return_length = True,return_tensors="pt")
        test_ids = test_input['input_ids'].to(device, dtype = torch.long)
        test_mask = test_input['attention_mask'].to(device, dtype = torch.long)
        test_token_type_ids = test_input['token_type_ids'].to(device, dtype = torch.long)
        test_lengths = test_input['length']#.squeeze(dim=1)#.to(device, dtype = torch.long)
        #print("shape",test_ids.shape, test_mask.shape, test_token_type_ids.shape, test_lengths)
        outputs = model(test_ids, test_mask, test_token_type_ids, test_lengths)
        _, output_idx = torch.max(outputs, dim =1)
        t_output_class = output_idx.cpu().detach().numpy().tolist()
        t_entity_label.append(t_output_class[0])
        # if t_output_class[0]==1:
        #   temp_list1.append(t_entity_ph[t5].lower())
    #print("pred list",temp_list1,t_entity_label)
    ### start triple formation from predicate
    for counterr, i1 in enumerate(zip(t_entity_ph, t_entity_label)):
      if (i1[1]==1):
        if ((counterr-1)>=0) and ((counterr+1)<len(t_entity_ph)):
          B_search = counterr-1
          while (B_search>=0):
            if (t_entity_label[B_search]==0):
              break
            else:
              B_search = B_search - 1
          L_search = counterr+1
          while (L_search < len(t_entity_ph)): # coorection should be less than 1
            if (t_entity_label[L_search]==0):
              break
            else:
              L_search = L_search + 1    
          if (B_search>=0) and (L_search<len(t_entity_ph)):
            triple_list = triple_list + [[t_entity_ph[B_search],i1[0],t_entity_ph[L_search]]]

    ### end triple formation
    output_info_folder = os.path.join(test_input_dir,fls,str(i),'triples')
    #print("list", os.listdir(test_input_dir + fls + '/' + str(i)+'/triples/'))
    info_applicable = [infoname_test_to_idx[t11] for t11 in os.listdir(output_info_folder) if t11 !='research-problem.txt' and t11 !='code.txt'] ## info indicies applicable to the document

    ### start triple classification into info unit
    with torch.no_grad():
      for t7 in range(len(triple_list)): #lowercase the string input to classifier
        test_in_trip = tokenizer_trip.encode_plus((' '.join(triple_list[t7])).lower(), None, add_special_tokens=True, 
                                           max_length= MAX_LEN_trip, pad_to_max_length=True, 
                                           return_token_type_ids=True, truncation=True, 
                                           return_length = True,return_tensors="pt")
        test_ids = test_in_trip['input_ids'].to(device, dtype = torch.long)
        test_mask = test_in_trip['attention_mask'].to(device, dtype = torch.long)
        test_token_type_ids = test_in_trip['token_type_ids'].to(device, dtype = torch.long)
        test_lengths = test_in_trip['length']#.squeeze(dim=1)#.to(device, dtype = torch.long)
        #print("shape",test_ids.shape, test_mask.shape, test_token_type_ids.shape, test_lengths)
        outputs_trip = model_trip(test_ids, test_mask, test_token_type_ids, test_lengths)
        #_, output_idx_trip = torch.max(outputs, dim =1)
        output_all_class = outputs_trip.cpu().detach().numpy().tolist()
        #print(t_output_class,'(' + '||'.join(t_triplet_formed[0])+')')
        output_app_class = [output_all_class[0][t12] if t12 in info_applicable else -10000 for t12 in range(len(output_all_class[0]))]
        
        output_file=open(os.path.join(output_info_folder, infoname_idx_to_test[output_app_class.index(max(output_app_class))]),"a")
        output_file.write('(' + '||'.join(triple_list[t7])+')'+"\n")
        # if output_app_class.index(max(output_app_class)) not in info_applicable:
        #   print("this", info_applicable, len(output_all_class[0]), output_app_class.index(max(output_app_class)))

        #for t7 in range(len(t_output_class)):
          #print(infoname_test_to_idx[t_output_class[t7]])
        # output_file=open(os.path.join(output_info_folder, infoname_idx_to_test[t_output_class[0]]),"a")
        # output_file.write('(' + '||'.join(t_triplet_formed[t6])+')'+"\n")
    ### end triple classification into info unit
  print("completed",fls)
  

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


completed constituency_parsing
completed coreference_resolution
completed data-to-text_generation
completed dependency_parsing
completed document_classification
completed entity_linking
completed face_alignment
completed face_detection
completed hypernym_discovery
completed natural_language_inference


In [25]:
triple_list

[['RC', 'develop', 'novel context zoom - in network ( ConZNet )'],
 ['novel context zoom - in network ( ConZNet )', 'for', 'RC tasks'],
 ['RC tasks', 'skip through', 'irrelevant parts'],
 ['irrelevant parts', 'of', 'document'],
 ['document', 'generate', 'answer'],
 ['answer', 'using', 'only the relevant regions of text'],
 ['ConZNet architecture', 'consists of', 'two phases'],
 ['two phases', 'In', 'first phase'],
 ['first phase', 'identify', 'relevant regions'],
 ['relevant regions', 'of', 'text'],
 ['text', 'by employing', 'reinforcement learning algorithm'],
 ['second phase', 'based on', 'encoder - decoder architecture'],
 ['identified regions of text', 'generates', 'answer'],
 ['answer', 'by using', 'residual self - attention network'],
 ['residual self - attention network', 'as', 'encoder'],
 ['RNNbased sequence generator', 'along with', 'pointer network'],
 ['pointer network', 'as', 'decoder'],
 ['ability', 'to select', 'relevant regions'],
 ['relevant regions', 'of', 'text'],
 [

In [29]:
test_input_dir = "/content/drive/MyDrive/sub5_ph22/"
# Test dataset reading
import os
from shutil import copyfile
test_list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
t_files = 0
for fls in test_list_of_folders:
  for i in os.listdir(test_input_dir + fls + '/'):
    if 'ablation-analysis.txt' in os.listdir(test_input_dir + fls + '/' + str(i)+'/triples'):
    #if 'model.txt' not in os.listdir(test_input_dir + fls + '/' + str(i)+'/triples') and 'approach.txt' not in os.listdir(test_input_dir + fls + '/' + str(i)+'/triples'):
      t_files=t_files+1
      print(fls,i)
    # for files in os.listdir(test_input_dir + fls + '/' + str(i)+'/triples'):
    #   if files.endswith("model.txt"):
    #     t_files=t_files+1
  #print("completed",fls,t_files)

constituency_parsing 1
coreference_resolution 8
coreference_resolution 4
data-to-text_generation 0
dependency_parsing 8
entity_linking 7
face_alignment 8
face_alignment 5
face_alignment 13
face_alignment 12
face_alignment 1
face_detection 6
face_detection 21
face_detection 7
face_detection 4
face_detection 15
face_detection 13
natural_language_inference 4
natural_language_inference 26
natural_language_inference 22
natural_language_inference 25


In [31]:
# Test dataset reading
import os
from shutil import copyfile
output_dir = "/content/drive/My Drive/sub9_p22/"
test1_input_dir = "/content/drive/My Drive/sub1_p22/"
test_list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
test1_file_name_list = []
t_files = 0
for fls in test_list_of_folders:
  count=0
  for i in os.listdir(output_dir + fls + '/'):
    count=count+1
    for files in os.listdir(output_dir + fls + '/' + str(i)+'/triples/'):
      if files.endswith("research-problem.txt"):
        #os.remove(os.path.join(output_dir,fls,str(i),'triples',files))
        #copyfile(test1_input_dir + fls + '/' + str(i) + '/' + files, output_dir + fls + '/' + str(i) + '/' + files)
        t_files=t_files+1
    test1_file_name_list.append(fls + '/' + str(i))
  print("completed sent",fls,count)
  print("completed entity",fls)

completed sent constituency_parsing 9
completed entity constituency_parsing
completed sent coreference_resolution 10
completed entity coreference_resolution
completed sent data-to-text_generation 7
completed entity data-to-text_generation
completed sent dependency_parsing 9
completed entity dependency_parsing
completed sent document_classification 21
completed entity document_classification
completed sent entity_linking 17
completed entity entity_linking
completed sent face_alignment 19
completed entity face_alignment
completed sent face_detection 22
completed entity face_detection
completed sent hypernym_discovery 9
completed entity hypernym_discovery
completed sent natural_language_inference 32
completed entity natural_language_inference
